# 2.4 Deep Learning

# Import Libraries and Data

In [4]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

2025-04-01 11:34:18.109847: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#Import unscaled weather dataset
df_weather = pd.read_csv('/Users/ariedlin/Desktop/Career Foundry Assignments/Machine Learning/2. Data/Supervised/weather_cleaned.csv')

In [6]:
df_weather.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9,1960-01-01
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1,1960-01-02
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9,1960-01-03
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6,1960-01-04
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4,1960-01-05


In [7]:
df_weather.shape

(22950, 136)

In [8]:
#Import Prediction Data Set
df_answers = pd.read_csv('/Users/ariedlin/Desktop/Career Foundry Assignments/Machine Learning/2. Data/Supervised/Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv')

In [9]:
df_answers.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
df_answers.shape

(22950, 16)

# Data Wrangling & Prep

In [20]:
#Drop DATE column in weather dataset
df_weather.drop(['DATE'], axis=1, inplace=True)

In [22]:
df_weather.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [24]:
#Drop DATE column in answers dataset
df_answers.drop(['DATE'], axis=1, inplace=True)

In [26]:
df_answers.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Shaping the Data for Modelling

In [43]:
#Set the X axis as array with weather data
X = np.array(df_weather)

In [45]:
#Set the y axis as array with answer data
y = np.array(df_answers)

In [47]:
#Ensure correct shapes
X.shape

(22950, 135)

In [49]:
y.shape

(22950, 15)

In [51]:
#Reshape X
X = X.reshape(-1,15,9)

In [53]:
#Ensure it worked
X.shape

(22950, 15, 9)

In [55]:
#Use argmax to transform y
y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0])

In [57]:
y.shape

(22950,)

In [59]:
#Check y layout
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

# Splitting the Data

In [62]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [64]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


# Bayesian Hyperparameter Optimization

In [67]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 #Number of weather stations
#Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [69]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [71]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
15/15 - 3s - 227ms/step - accuracy: 0.6001 - loss: 2.7105
Epoch 2/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 1s - 87ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 1s - 88ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 1s - 88ms/step - accuracy: 0.6440 - loss: 2.6894
Epoch 7/25
15/15 - 1s - 87ms/step - accuracy: 0.6440 - loss: 2.6873
Epoch 8/25
15/15 - 1s - 88ms/step - accuracy: 0.6440 - loss: 2.6853
Epoch 9/25
15/15 - 1s - 88ms/step - accuracy: 0.6440 - loss: 2.6835
Epoch 10/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6817
Epoch 11/25
15/15 - 1s - 88ms/step - accuracy: 0.6440 - loss: 2.6800
Epoch 12/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6784
Epoch 13/25
15/15 - 1s - 95ms/step - accuracy: 0.6440 - loss: 2.6768
Epoch 14/25
15/15 - 1s - 92ms/step - accuracy: 0.6440 - loss: 2.6753
Epoch 15/25
15/15 - 1s - 87ms/step - accur

/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 - 3s - 201ms/step - accuracy: 0.5992 - loss: 2.7168
Epoch 2/25
15/15 - 1s - 93ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6943
Epoch 5/25
15/15 - 1s - 90ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 1s - 92ms/step - accuracy: 0.6440 - loss: 2.6895
Epoch 7/25
15/15 - 1s - 88ms/step - accuracy: 0.6440 - loss: 2.6874
Epoch 8/25
15/15 - 1s - 91ms/step - accuracy: 0.6440 - loss: 2.6854
Epoch 9/25
15/15 - 1s - 91ms/step - accuracy: 0.6440 - loss: 2.6836
Epoch 10/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6818
Epoch 11/25
15/15 - 1s - 88ms/step - accuracy: 0.6440 - loss: 2.6801
Epoch 12/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6785
Epoch 13/25
15/15 - 1s - 91ms/step - accuracy: 0.6440 - loss: 2.6769
Epoch 14/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6754
Epoch 15/25
15/15 - 1s - 87ms/step - accuracy: 0.6440

...

/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 2s - 109ms/step - accuracy: 0.5325 - loss: 1.5846
Epoch 2/35
21/21 - 1s - 30ms/step - accuracy: 0.6602 - loss: 0.9757
Epoch 3/35
21/21 - 1s - 30ms/step - accuracy: 0.6954 - loss: 0.8810
Epoch 4/35
21/21 - 1s - 30ms/step - accuracy: 0.7166 - loss: 0.8179
Epoch 5/35
21/21 - 1s - 30ms/step - accuracy: 0.7311 - loss: 0.7707
Epoch 6/35
21/21 - 1s - 31ms/step - accuracy: 0.7445 - loss: 0.7384
Epoch 7/35
21/21 - 1s - 33ms/step - accuracy: 0.7531 - loss: 0.7066
Epoch 8/35
21/21 - 1s - 30ms/step - accuracy: 0.7644 - loss: 0.6790
Epoch 9/35
21/21 - 1s - 30ms/step - accuracy: 0.7726 - loss: 0.6510
Epoch 10/35
21/21 - 1s - 30ms/step - accuracy: 0.7846 - loss: 0.6182
Epoch 11/35
21/21 - 1s - 30ms/step - accuracy: 0.7911 - loss: 0.5912
Epoch 12/35
21/21 - 1s - 30ms/step - accuracy: 0.8023 - loss: 0.5620
Epoch 13/35
21/21 - 1s - 30ms/step - accuracy: 0.8109 - loss: 0.5350
Epoch 14/35
21/21 - 1s - 30ms/step - accuracy: 0.8188 - loss: 0.5105
Epoch 15/35
21/21 - 1s - 30ms/step - accuracy: 0.8277

/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 3s - 121ms/step - accuracy: 0.5099 - loss: 1.6275
Epoch 2/35
21/21 - 1s - 31ms/step - accuracy: 0.6558 - loss: 1.0299
Epoch 3/35
21/21 - 1s - 31ms/step - accuracy: 0.6805 - loss: 0.9378
Epoch 4/35
21/21 - 1s - 31ms/step - accuracy: 0.7015 - loss: 0.8663
Epoch 5/35
21/21 - 1s - 30ms/step - accuracy: 0.7237 - loss: 0.8055
Epoch 6/35
21/21 - 1s - 30ms/step - accuracy: 0.7378 - loss: 0.7568
Epoch 7/35
21/21 - 1s - 30ms/step - accuracy: 0.7497 - loss: 0.7170
Epoch 8/35
21/21 - 1s - 30ms/step - accuracy: 0.7630 - loss: 0.6805
Epoch 9/35
21/21 - 1s - 30ms/step - accuracy: 0.7752 - loss: 0.6469
Epoch 10/35
21/21 - 1s - 30ms/step - accuracy: 0.7848 - loss: 0.6155
Epoch 11/35
21/21 - 1s - 30ms/step - accuracy: 0.7993 - loss: 0.5792
Epoch 12/35
21/21 - 1s - 30ms/step - accuracy: 0.8052 - loss: 0.5502
Epoch 13/35
21/21 - 1s - 30ms/step - accuracy: 0.8095 - loss: 0.5338
Epoch 14/35
21/21 - 1s - 30ms/step - accuracy: 0.8203 - loss: 0.5032
Epoch 15/35
21/21 - 1s - 30ms/step - accuracy: 0.8284

ValueError: Input y contains NaN.

In [73]:
#Check y array to see if there are NaN values
print(np.isnan(y).sum()) 

0


It looks like the algorithm may have enacted “early stopping” because it’s not improving on the results.  When it stopped, it was at 92.6% accuracy and .211 loss which is pretty good without overfitting.

In [76]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 47,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x15f02ef00>}

In [80]:
#Set the model with optimized hyperparameters

epochs = 47
batch_size = 460

timesteps = len(X_train[0])  
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  
neurons = 61
normalization = 0.770967179954561  
dropout = 0.7296061783380641  
dropout_rate = 0.19126724140656393  
optimizer = Adadelta(learning_rate=0.7631771981307285)  

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

# Add second set of dense layers
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())  # Max-pooling layer to downsample the data
model.add(Flatten())  # Flatten for the dense layers
model.add(Dense(n_classes, activation='softmax'))  # Output layer for classification

# Compile the model with specified loss and optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [82]:
#Show the summary
model.summary()

Model: "sequential_75"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_75 (Conv1D)              │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_380 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_381 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_382 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_75 (MaxPooling1D) │ (None, 7, 61)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_75 (Flatten)            │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_383 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,169 (74.88 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

In [84]:
#Put the y_test set back into a one-hot configuration
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [86]:
#Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [88]:
#Compile the model with categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [90]:
#Fit the model to the data
model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/47
38/38 - 3s - 84ms/step - accuracy: 0.5987 - loss: 1.3950
Epoch 2/47
38/38 - 1s - 35ms/step - accuracy: 0.6969 - loss: 0.9088
Epoch 3/47
38/38 - 1s - 34ms/step - accuracy: 0.7252 - loss: 0.8112
Epoch 4/47
38/38 - 1s - 34ms/step - accuracy: 0.7423 - loss: 0.7567
Epoch 5/47
38/38 - 1s - 34ms/step - accuracy: 0.7613 - loss: 0.7079
Epoch 6/47
38/38 - 1s - 34ms/step - accuracy: 0.7752 - loss: 0.6722
Epoch 7/47
38/38 - 1s - 34ms/step - accuracy: 0.7843 - loss: 0.6351
Epoch 8/47
38/38 - 1s - 35ms/step - accuracy: 0.7953 - loss: 0.6039
Epoch 9/47
38/38 - 1s - 34ms/step - accuracy: 0.8022 - loss: 0.5794
Epoch 10/47
38/38 - 1s - 34ms/step - accuracy: 0.8107 - loss: 0.5538
Epoch 11/47
38/38 - 1s - 34ms/step - accuracy: 0.8206 - loss: 0.5342
Epoch 12/47
38/38 - 1s - 38ms/step - accuracy: 0.8224 - loss: 0.5147
Epoch 13/47
38/38 - 1s - 34ms/step - accuracy: 0.8304 - loss: 0.4990
Epoch 14/47
38/38 - 1s - 35ms/step - accuracy: 0.8370 - loss: 0.4749
Epoch 15/47
38/38 - 1s - 34ms/step - accura

# Confusion Matrix

In [93]:
#Define list of stations names
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [95]:
print("Unique classes in y_test:", np.unique(y_test))

Unique classes in y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]


In [97]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [99]:
#Before making predictions, convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test, num_classes=15)

In [101]:
#Before making predictions, convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test, num_classes=15)

#Predict the class probabilities
y_pred = model.predict(X_test)

#Convert y_test and y_pred to class labels
if y_test_one_hot.ndim == 1:
    y_test_labels = y_test_one_hot
else:
    y_test_labels = np.argmax(y_test_one_hot, axis=1)

if y_pred.ndim == 1:
    y_pred_labels = y_pred
else:
    y_pred_labels = np.argmax(y_pred, axis=1)

#Manually calculate accuracy
correct_predictions = np.sum(y_test_labels == y_pred_labels)
total_samples = len(y_test_labels)
accuracy = correct_predictions / total_samples

print(f'Accuracy: {accuracy * 100:.2f}%')

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 87.96%


In [103]:
y_pred_labels = np.argmax(y_pred, axis=1)

In [105]:
#Convert y_test_one_hot back to class labels
y_test_labels = np.argmax(y_test_one_hot, axis=1)

In [107]:
#Evaluate
print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3482       117        27       3           7         9       2   
BELGRADE       86       854        91       5          11         9       0   
BUDAPEST       35         2       144      13           5         7       1   
DEBILT         12         0         4      50           8         6       2   
DUSSELDORF      3         1         0       1          10         9       0   
HEATHROW        7         1         1       0           3        55       0   
KASSEL          1         0         0       0           0         0       5   
LJUBLJANA       9         1         1       0           0         0       0   
MAASTRICHT      5         0         0       0           0         1       0   
MADRID         44         4         7       1           0         5       1   
MUNCHENB        5         0         0       0       

In [109]:
print("Unique values in y_test:", np.unique(y_test_labels))
print("Unique values in y_pred:", np.unique(y_pred_labels))

Unique values in y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
Unique values in y_pred: [ 0  1  2  3  4  5  6  7  8  9 10 11]


This decision maxtix looks pretty good, but there are 3 stations not being recognized. 